---

# CSCI 3202, Fall 2022
# Homework 4
# Due: Friday September 30, 2022 at 6:00 PM

<br> 

### Your name: Giuliano Costa

<br> 

---

Some useful packages and libraries:




In [1]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter


---

# Hidden Markov Models
In this assignment, you will build a Hidden Markov Model from a data set to detect typos in text. One major difference between this assignment and previous assignments in this class is we're not providing any skeleton code, you need to generate your code from scratch. You can use any code provided during lecture or on previous assignments. 
<p><strong>Please read the entire assignment before beginning.</strong> 
    
<p>The main components of the assignment are the following:

### Part I - Building an HMM from data
1.  Implement a method to parse the data file and build an HMM from the data.
2.  Implement a method, Laplace smoothing, to smooth the transition and emission probabilities

### Part II - Implement the Viterbi algorithm
1.	Implement the Viterbi algorithm for finding the most likely sequence of states through the HMM, given "evidence"; and
2.	Run your code on a separate dataset and explore its performance.
    
### Part III - Summarize your results
    



## Part 1 - Building an HMM from data
The first part of the assignment is to build an HMM from data. Recall that an HMM involves a hidden state that changes over time, as well as observable evidence that is used to infer the hidden state. In our example from lecture, we used the observation of how a patient felt to infer whether they were healthy or sick. 

An HMM is defined by three sets of probabilities:
1.	For each state s, the probability of observing each output o at that state (P(E[t]=o | X[t]=s). These are the emission probabilities.
2.	From each state s, the probability of traversing to every other state s' in one time step (P(X[t+1]=s' | X[t]=s)). These are the transition probabilities.
3.	A distribution over the start state (P(X[0])). This is the initial state distribution.

For the start state distribution (P(X[0])), in this assignment, you will assume that there is a single dummy start state, distinct from all other states, and to which the HMM can never return.  When you implement the Viterbi algorithm in Part II, you will need to include the probability of making a transition from this dummy start state to each of the other states.

### Generate the conditional probability tables
To generate the transition and emission probability tables, use the data file called <em>typos20.data</em> on Canvas that contains sequences of state-output pairs, i.e., sequences of the form: <br>

x[1] e[1] <br>
x[2] e[2] <br>
. <br>
. <br>
. <br>
x[n] e[n]. <br>  

In this data, the x[i] is the state and the e[i] is the observation at time i. 

For instance, to estimate the probability of output o being observed in state s, you count the number of times that output o appears with state s in the given data, and divide by a normalization constant (so that the probabilities of all outputs from that state add up to one).  In this case, that normalization constant would simply be the number of times that state s appears at all in the data.

In this problem, state is letter that should have been typed, and output is the letter that was typed.  Given a sequence of outputs (i.e., actually typed letters), the problem is to reconstruct the hidden state sequence (i.e., the intended sequence of letters).  Thus, data for this problem looks like this:

i i <br>
n n <br>
t t <br>
r r <br>
o o <br>
d x <br>
u u <br>
c c <br>
t t <br>
i i <br>
o i <br>
n n <br>
_ _ <br>
t t <br>
h h <br>
e e <br>
_ _ <br>

where the left column is the correct text and the right column contains text with errors.

### Data
Data for this assignment was generated by starting with a text document, in this case, the Unabomber's Manifesto, which was chosen not for political reasons, but for its convenience being available on-line and of about the right lens, converting all numbers and punctuation to white space and converting all letters to lower case.  The remaining text is a sequence only over the lower case letters and the space character, represented in the data files by an underscore character.  Next, typos were artificially added to the data as follows: with 80% probability, the correct letter is transcribed, but with 20% probability, a randomly chosen neighbor (on an ordinary physical keyboard) of the letter is transcribed instead.  Space characters are always transcribed correctly. 

As an example, the original document begins:

<em>introduction the industrial revolution and its consequences have been a disaster for the human race they have greatly increased the life expectancy of those of us who live in advanced countries but they have destabilized society have made life unfulfilling have subjected human beings to indignities have led to widespread psychological suffering in the third world to physical suffering as well and have inflicted severe damage on the natural world the continued development of technology will worsen the situation it will certainly subject human beings to greater indignities and inflict greater damage on the natural world it will probably lead to greater social disruption and psychological suffering and it may lead to increased physical suffering even in advanced countries the industrial technological system may survive or it may break down if it survives it may eventually achieve a low level of physical and psychological suffering but only after passing through a long and very painful period of adjustment and only at the cost of permanently reducing human beings and many other living organisms to engineered products and mere cogs in the social machine </em>

With 20% noise, it looks like this:

<em>introductipn the industfial revolhtjon and its consequences bafw newn a diszster rkr the yumab race thdy have grwatky increased the ljte esoectandy od thosr of is who libe in advanced coubfries but they have fewtabipuzee xociwty have made life ujfuorillkng have wubjwdted humah beints to incihbjtids have led to qidespreze lsyxhllotical shffeding kn tne third wkrld to phyxicql sufcefimg as weol and hqve ingoidtex srvere damsge on the natural world the confinued developmeng of twvhjllogy will wotsen thd situation it wull certaknly sunjrct yyman beingw tl greater ibdignities snd infpixt greagwr damsge on fhe natural alrld it wjlk probably lwad tk grezter sofiqp disrupgiln and pstchokofucal wufterkng anc it may kead fl uncreqxed pgusiczl sucfreinh even in acgajved countries the indhsteial tedhnologicak system may survivr or ut nay brezk down uf it survives it nay evenyuakly achieve a los lwvel of phyxkcal and psycyoligical sufveribg but only after passing theough a long amd very painful periox od adjuwtmebt and only at the fost kf permsnently reducing hymaj veings abs nsjy otgwr kuving orbanisms to envineered leoduxfs amd mere clgs in thr soxiap maxhjne </em>

The error rate (fraction of characters that are mistyped) is about 16.5% (less than 20% because space characters were not corrupted).

### Implement Laplace smoothing
When you generate the transition and emission probabilities, you will also need to smooth the estimates.  Here's the rationale, consider flipping a coin for which the probability of heads is p, where p is unknown, and our goal is to estimate p. We count how many times the coin comes up heads and divide by the total number of coin flips.  If we flip the coin 1000 times and it comes up heads 367 times, it is very reasonable to estimate p as approximately 0.367.  However, suppose we flip the coin only twice and we get heads both times.  Is it reasonable to estimate p as 1.0?  Intuitively, given that we only flipped the coin twice, we don’t have enough data to conclude that the coin will always come up heads. Smoothing is a way of avoiding these types of conclusions when we have limited data available.  

A simple smoothing method, called Laplace smoothing (or Laplace's law of succession or add-one smoothing in your textbook), is to estimate p by:

<p>

$$\frac{(1+number of heads)}{(2+number of flips)}$$

We add 1 to each output that could be observed in the numerator, and add the total number of states to the denominator.

If we are keeping count of the number of heads and the number of tails, this rule is equivalent to starting each of our counts at one, rather than zero.  This latter view generalizes to the case in which there are more than two possible outcomes (for instance, estimating the probability of a die coming up on each of its six faces). 
Another advantage of Laplace smoothing is that it avoids estimating any probabilities to be zero, even for events never observed in the data.  For HMMs, this is important since zero probabilities can be problematic for some algorithms.
    
For this assignment, you should use Laplace-smoothed estimates of probabilities.  For instance, returning to the problem of estimating the probability of output o being observed in state s, you would use one plus the number of times output o appears in state s in the given data, divided by a normalization constant. In this case, the normalization constant would be the number of times state s appears in the data, plus the total number of possible outputs. You will need to also work out Laplace-smoothed estimates for item 2, i.e., for the probability of making a transition from one state to another, as well as the probability of making a transition from the dummy start state to any of the other states. 


In [2]:
## Your code needs to display the first 10 lines of your emission and transition tables
## and generate an output file with the complete tables, called Assignment8Output_Lastname_Firstname
# Your code also needs to be structured as an HMM class

In [3]:
""" 
class HMM
Inputs: a processed dataframe containing a state and output sequence of letters.
Outputs: Emission, Transition, and Initialization Tables. Saved as a CSV.
"""

class HMM():
    def __init__(self, typos_df):
        
        #### Initial Distribution ####
        states = typos_df['State']
        state_sums = {} 
        total = 0
        letters = ''.join(sorted(set(states)))
        
        for i in letters:
            state_sums[i] = states.str.count(i).sum()
            total += state_sums[i]
        #Sum state letter counts and divide by sum of letters
        a = {k: v / total for k, v in state_sums.items()}        
        init_distribution = pd.DataFrame.from_dict(a, orient="index", columns=['State'])        
        
        #### Transition Matrix ####
        #S to S'
        # Create a raw transition matrix
        #Take the state column and shift it down by one index
        state = typos_df['State'].to_numpy()
        state_prime = typos_df['State'].to_numpy()
        state_prime = np.roll(state_prime, -1, axis=0)

        a = np.column_stack((state, state_prime))

        matrix = pd.Series(Counter(map(tuple, a))).unstack().fillna(0)
        matrix = matrix.rename_axis("S", axis="rows")
        matrix = matrix.rename_axis("S'", axis="columns")

        # Normalize the rows according to the laplace smoothing.
        denom = matrix
        numerator = matrix + 1
        transition_probs = numerator.divide(denom.sum(axis=1) + 27, axis=0)
        self.trans = transition_probs

        #### Emission ####
        a = typos_df.to_numpy()
        matrix = pd.Series(Counter(map(tuple, a))).unstack().fillna(0)

        matrix = matrix.rename_axis("State", axis="rows")
        matrix = matrix.rename_axis("Output", axis="columns")

        # Normalize the rows according to the laplace smoothing.
        denom = matrix
        numerator = matrix + 1
        emission_probs = numerator.divide(denom.sum(axis=1) + 27, axis=0)
        self.em = emission_probs
        self.init = init_distribution
        self.letters = letters
        
    def tables(self, print_tables):
        pd.concat([
        pd.concat([self.init, self.trans], axis=1),
        pd.concat([self.trans, self.em], axis=0)]).to_csv('./data/HW4_output_Costa_Giuliano.csv')

        if(print_tables == True):
            display("Transition Matrix:", self.trans.head(10))
            display("Emission Matrix:", self.em.head(10))        
        return self.init, self.trans, self.em



## Part II - Viterbi - finding the most likely sequence

Once your CPTs for your HMM are built, the second part of the assignment is to implement the Viterbi algorithm and use your CPTs to compute the most probable sequence of states (according to the HMM that you built from the data) for a given sequence of outputs.

There is another file on Canvas called *typos20Test.data* that contains test sequences of state-output pairs. You will provide your Viterbi code with just the output part of each of these sequences, and from this, you must compute the most likely sequence of states to produce the output sequence. The state part of these sequences is provided so that you can compare the estimated state sequences generated by your code to the actual state sequences that generated this data.  Note that these two sequences will not necessarily be identical, even if you have correctly implemented the Viterbi algorithm.

A numerical tip: the Viterbi algorithm involves multiplying many probabilities together.  Since each of these numbers is smaller than one (possibly much smaller), you can end up working with numbers that are tiny enough to be computationally indistinguishable from zero. To avoid this problem, it is recommended that you work with log probabilities. To do so, rather than multiplying two probabilities p and q, simply add their logarithms using the rule:
<p>
<center> log(pq) = log(p) + log(q) </center>

You will probably find that there is never a need to store or manipulate actual probabilities; instead, everything can be done using log probabilities.
The text reconstructed using an HMM with the Viterbi algorithm looks like this:
    <p>
*introduction the industrial revoluthon and its consequences bare neen a dissster ror the tuman race they have greatly increased the lite esoectandy od those of is who libe in advanced counfries but they have festabupusee cocisty have made live intiorilling have wibjested human beints to incingitids have led to widesprese lsysullotical suffeding in the third world to physical surcefing as weol and have ingoistes severe damage on the natural world the continued developmeng of techillogy will wotsen the situation it will certaknly sunirct tyman beinge tl greater indithities and infoist greager damage on the natural aleld it will probably owad to grester sofial distuption and pstchomofucal wiftering and it may kead fl increqxed ogusical suctreing even in achanved countries the industeial technologicak system may survive or ut nay break down if it survives it nay eventually achieve a los level of physical and psycholigical survering but only arter passing theough a long and very paindul perios od adjustment and only at the fost of permanently reducing human veings ans nany other kiving organisms to envineered leodusts and mere clys in the social machine*
        
The error rate has dropped to about 10.4%.


In [4]:
""" Your code goes here. You need to implement the viterbi algorithm, and display the output that your code generates
    You also need to display the error rate for your algorithm.
    
    Inspiration taken from a chord matching algorithm made by an audio production school:
    https://www.audiolabs-erlangen.de/resources/MIR/FMP/C5/C5S3_Viterbi.html 
"""

train_df = pd.read_csv("./data/typos20.data", sep=' ', header = None, names=["State", "Output"])
test_df = pd.read_csv("./data/typos20Test.data", sep=' ', header = None, names=["State", "Output"])

"""
    Inputs:
        O: Observation Space O = {O1, ... On }
        S: State Space S = {S1, ... Sn}
        Pi: Initital Probabilities
        Y: Sequence of Observations
        A: Transition Matrix K x K
        B: Emission Matrix K x N
   Returns:
         S_opt (np.ndarray): Optimal state sequence, where integer matches to possible state letter
         D_log (np.ndarray): Accumulated log probability matrix
         E (np.ndarray):     Backtracking matrix
            
"""

def Viterbi(O, pi, Tn, Em, HMM):
    S_num = Tn.shape[0] #Possible states
    O_num = O.shape[0] #Observation Sequence Length
    Tn_log = np.log(Tn.to_numpy()) #A Transition
    Em_log = np.log(Em.to_numpy()) #B Emission
    Pi_log = np.log(pi.to_numpy()) #C Initital

    #Initialize backtracking and Optimal State matrix
    E = np.zeros((S_num, O_num-1)).astype(np.int32)
    D = np.zeros((S_num, O_num)) #Possible states I x N
    
    D[:, [0]] = Em_log[:,[0]] + Pi_log #Add the log probabilities using log(A*B) = log(A) + log(B)

    O = O.to_numpy()
        
    for n in range(0, O_num):   #for all Outputs
        for i in range(S_num):  #for all Possible States 
            temp_sum = Tn_log[:, i] + D[:, n - 1]
            D[i, n] = np.max(temp_sum) + Em_log[i, HMM.letters.index(O[n])]
            E[i, n-1] = np.argmax(temp_sum) #set previous column to maxes of temp_sum

    #Backtracking
    S_opt = np.zeros(O_num).astype(np.int32) 
    S_opt[-1] =  np.argmax(D[:, -1])
    for n in range(O_num - 2, -1, -1): #For the observation length, set the optimal sequence equal to backtracking results.
         S_opt[n] = E[int(S_opt[n + 1]), n] 
    
    return S_opt, D, E
    
### GET TABLES using HMM using only the TRAINING dataset. ###
train_test = HMM(train_df)
init, trans, em = train_test.tables(print_tables=False)

#### GET Output sequence from only the TESTING dataset. ###
O = test_df['Output']

### COMPUTER VITERBI ###
S_opt, D, E = Viterbi(O, init, trans, em, train_test)

### CONVERT Optimal state sequence to letters ###
S = test_df['State']
hidden_state = []
for i in S_opt:
    hidden_state.append(train_test.letters[i])
    
### Calculate Error ###    
correct = 0
for i in range(len(S_opt)):
    if(train_test.letters[S_opt[i]] == S[i]):
        correct += 1

print("Observation sequence:   ", ''.join(O.to_numpy()[50:100]), "")
print("Optimal state sequence: ", ''.join(hidden_state[50:100]), "")
print("Actual state sequence:  ", ''.join(S.values[50:100]), "\n")
print("Accuracy of Optimal State Sequence: %", round((correct/len(S_opt)) * 100 ,4))

Observation sequence:    sequences_bafw_newn_a_diszster_rkr_the_yumab_race_ 
Optimal state sequence:  sequences_bare_neen_a_dissster_ror_the_tuman_race_ 
Actual state sequence:   sequences_have_been_a_disaster_for_the_human_race_ 

Accuracy of Optimal State Sequence: % 89.5684


In [5]:
display_tables = train_test.tables(print_tables=True)

'Transition Matrix:'

S',_,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
S,,,,,,,,,,,,,,,,,,,,,
_,0.000037,0.105182,0.049504,0.041529,0.028997,0.030871,0.035943,0.016869,0.035465,0.080632,...,0.001213,0.026204,0.075303,0.174789,0.010217,0.004814,0.053804,0.000147,0.002756,0.000037
a,0.067629,0.000101,0.022509,0.039063,0.028364,0.000202,0.004239,0.020188,0.000505,0.027354,...,0.000303,0.090138,0.064702,0.166751,0.016150,0.033108,0.002826,0.000505,0.024932,0.000707
b,0.009625,0.037994,0.002533,0.001013,0.000507,0.407801,0.001013,0.000507,0.000507,0.036981,...,0.000507,0.035968,0.008105,0.008612,0.126646,0.004053,0.000507,0.000507,0.096251,0.000507
c,0.034156,0.159982,0.000220,0.015425,0.000220,0.141913,0.000220,0.000220,0.162847,0.136844,...,0.002644,0.034156,0.002865,0.094976,0.028427,0.000220,0.000220,0.000220,0.005068,0.000220
d,0.473206,0.021901,0.000233,0.000699,0.004427,0.175676,0.000466,0.001864,0.000466,0.099021,...,0.000233,0.019571,0.034017,0.000233,0.056850,0.013979,0.001398,0.000233,0.009320,0.000233
e,0.329493,0.035567,0.001063,0.045197,0.057486,0.026882,0.017842,0.005613,0.003899,0.014357,...,0.003899,0.104632,0.087321,0.032435,0.000591,0.024105,0.004372,0.014179,0.012643,0.000059
f,0.371127,0.043117,0.000319,0.001916,0.000319,0.063877,0.062280,0.000639,0.000319,0.085596,...,0.000319,0.071543,0.001597,0.049505,0.038326,0.000319,0.000319,0.000319,0.014372,0.000319
g,0.270963,0.078028,0.000388,0.000388,0.000388,0.147516,0.000388,0.011258,0.079969,0.098214,...,0.000388,0.088121,0.024845,0.001553,0.026398,0.000388,0.000388,0.000388,0.062112,0.000388
h,0.092325,0.199269,0.001430,0.000159,0.000159,0.421738,0.000159,0.000159,0.000318,0.093596,...,0.000159,0.015891,0.003178,0.012236,0.033688,0.000159,0.000477,0.000159,0.008422,0.000159


'Emission Matrix:'

Output,_,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
State,,,,,,,,,,,,,,,,,,,,,
_,0.999044,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,...,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037,0.000037
a,0.000101,0.794792,0.000101,0.000101,0.000101,0.000101,0.000101,0.000101,0.000101,0.000101,...,0.068235,0.000101,0.068840,0.000101,0.000101,0.000101,0.000101,0.000101,0.000101,0.065812
b,0.000507,0.000507,0.798886,0.000507,0.000507,0.000507,0.000507,0.046099,0.043566,0.000507,...,0.000507,0.000507,0.000507,0.000507,0.000507,0.047112,0.000507,0.000507,0.000507,0.000507
c,0.000220,0.000220,0.000220,0.793521,0.044954,0.000220,0.052887,0.000220,0.000220,0.000220,...,0.000220,0.000220,0.000220,0.000220,0.000220,0.053768,0.000220,0.050022,0.000220,0.000220
d,0.000233,0.000233,0.000233,0.041007,0.801025,0.040541,0.033784,0.000233,0.000233,0.000233,...,0.000233,0.000233,0.040541,0.000233,0.000233,0.000233,0.000233,0.038211,0.000233,0.000233
e,0.000059,0.000059,0.000059,0.000059,0.066348,0.800662,0.000059,0.000059,0.000059,0.000059,...,0.000059,0.064812,0.000059,0.000059,0.000059,0.000059,0.066820,0.000059,0.000059,0.000059
f,0.000319,0.000319,0.000319,0.033536,0.030981,0.000319,0.802619,0.036091,0.000319,0.000319,...,0.000319,0.029064,0.000319,0.032577,0.000319,0.028745,0.000319,0.000319,0.000319,0.000319
g,0.000388,0.000388,0.041925,0.000388,0.000388,0.000388,0.041537,0.800466,0.030668,0.000388,...,0.000388,0.000388,0.000388,0.041149,0.000388,0.036102,0.000388,0.000388,0.000388,0.000388
h,0.000159,0.000159,0.043699,0.000159,0.000159,0.000159,0.000159,0.039886,0.791038,0.000159,...,0.000159,0.000159,0.000159,0.000159,0.000159,0.000159,0.000159,0.000159,0.037979,0.000159


## Part III - Summarize your results
As part of this assignment, include a short report in your notebook describing your work. The report should contain the following:
### Purpose: 

This homework essentially uses a training and testing data set of typos. The training set can be thought of the HMM tables which are generated
knowing the correct 'state' and a slightly incorrect 'output'. The purpose is to use the trained HMM model in conjunction with the dynamic programming Viterbi algorithm. Otherwise, this problem would be far too computationally difficult. The goal is to accurately predict the text using an incorrectly typed sequence. 

### Procedure:

##### Transition Probabilities

The transition probability is an N x N matrix of cumulative sums of going from one state to the subsequent state in a sequence of states. In our case, a long string of letters. Where each letter is a state S and the next state S'. I created the transition table using a Pandas dataframe. I simply shifted over the 'state' column by 1, to emulate the next state. Then, I created a dataframe from tuples which counts state transitions. This matrix was then Laplace smoothed.  

##### Emission Probabilities
Also using a Pandas dataframe, I create an O x S matrix, where N is the count of unique characters. Similar to the transition matrix, this was a cumulative sum of tuples. Where the columns are the possible states, and rows are possible outputs. Each cell is the count of state -> output for all possible states. This was then Laplace smoothed.

##### Viterbi Algorithm

After creating the HMM with Pandas, I converted all my tables to numpy arrays for simplicity. Two matrices are initialized, one which saves the max probability D (Optimal State Matrix) for each observation, and one which saves the indices of those optimal states, E. The D matrix is of shape I x N. Where the rows are the 27 possible states, and N columns are the observation length (~20,000). Tables were then converted to log tables. The 1st column of the D matrix is set to the sum of the Pi and Emission matrix.

In a nested loop, for each ouput and each possible state, the max probability is saved temporarily and input to D using transition and emission matrices. The indicies are saved in the backtracking matrix E. 

In another reverse loop, the backtracking indices are converted to the predicted state sequence. Since the state sequence is saved as integers (1-27), it was then simply converted back to a list of letters.  

### Data
The first dataset typos20.data was used as a training set for the HMM model. Whereas the typos20Test.data output column was only exposed to the Viterbi algorithm. The typos20Test.data state column (true sequence) was used to compare to the optimal state sequence output by Viterbi. The only preprocessing done to the data was conversion to dataframes of two columns, state and output. 

### Results

My Viterbi algorithm correctly predicted the state sequence with an error rate of 10.4%. Which makes sense as there are still typos in the sequence. I tested using the entire test dataset, and used the entire training dataset for the HMM. The optimal sequence incorrectly predicted certain mispelled words. Using Numpy and Pandas, this algorithm accurately predicts the sequence with a runtime of under 7 seconds.

